In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(work_dir+'/fgc_support_retri/')
from fgc_support_retri.train import *

I0313 02:01:05.717243 140613488523072 file_utils.py:39] PyTorch version 1.1.0 available.


In [3]:
train_sgroup_model(10, 12, "20200312_sgroup_sfscore_lr=2e-5")

I0313 02:01:07.651908 140613488523072 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0313 02:01:08.580983 140613488523072 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0313 02:01:08.583907 140613488523072 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

start training ... 


/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 0/247 [00:00<?, ?it/s]

epoch 0 train_loss: 0.221


100%|██████████| 247/247 [06:02<00:00,  1.60s/it]


...

{'sp_em': 0.08786610878661087, 'sp_prec': 0.5111017875662226, 'sp_recall': 0.6021518230723252, 'sp_f1': 0.5013441435912518}
epoch 9 eval_recall: 0.602 eval_f1: 0.501


  0%|          | 0/247 [00:00<?, ?it/s]

epoch 10 train_loss: 0.004


100%|██████████| 247/247 [05:49<00:00,  1.40s/it]


{'sp_em': 0.08786610878661087, 'sp_prec': 0.5111017875662226, 'sp_recall': 0.6021518230723252, 'sp_f1': 0.5013441435912518}
epoch 10 eval_recall: 0.602 eval_f1: 0.501
